# Import libraries

In [110]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers

In [111]:
folder_path = './Tank1'

file_list = glob.glob(folder_path + '/*.xlsx')

dfs = []

for file in file_list:
    df = pd.read_excel(file, parse_dates=['created_at'])
    dfs.append(df)
    
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.head()

,created_at,entry_id,field1
0,2018-02-14 04:50:55,1,40
1,2018-02-14 04:51:33,2,39
2,2018-02-14 04:52:12,3,40
3,2018-02-14 04:52:51,4,40
4,2018-02-14 04:53:30,5,40


# Convert the date column

In [112]:
merged_df["created_at"] = pd.to_datetime(merged_df["created_at"], format='%d/%m/%Y %H:%M')

merged_df["year"] = merged_df["created_at"].dt.year
merged_df["month"] = merged_df["created_at"].dt.month
merged_df["day"] = merged_df["created_at"].dt.day
merged_df["hour"] = merged_df["created_at"].dt.hour
merged_df["minute"] = merged_df["created_at"].dt.minute
merged_df.head()

,created_at,entry_id,field1,year,month,day,hour,minute
0,2018-02-14 04:50:55,1,40,2018,2,14,4,50
1,2018-02-14 04:51:33,2,39,2018,2,14,4,51
2,2018-02-14 04:52:12,3,40,2018,2,14,4,52
3,2018-02-14 04:52:51,4,40,2018,2,14,4,52
4,2018-02-14 04:53:30,5,40,2018,2,14,4,53


In [113]:
null_data = merged_df.isna().sum()
duplicate_rows = merged_df.duplicated()
print('Nulls', null_data)
print('Duplicates', duplicate_rows)

Nulls created_at    0
entry_id      0
field1        0
year          0
month         0
day           0
hour          0
minute        0
dtype: int64
Duplicates 0        False
1        False
2        False
3        False
4        False
         ...  
89392    False
89393    False
89394    False
89395    False
89396    False
Length: 89397, dtype: bool


# Check for outliers


In [114]:
# Calculate quartiles and IQR
Q1 = merged_df["field1"].quantile(0.25)
Q3 = merged_df["field1"].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify and remove outliers
df_no_outliers = merged_df[
    (merged_df["field1"] >= lower_bound) &
    (merged_df["field1"] <= upper_bound)
]

# Print the DataFrame without outliers
print(df_no_outliers)


               created_at  entry_id  field1  year  month  day  hour  minute
0     2018-02-14 04:50:55         1      40  2018      2   14     4      50
1     2018-02-14 04:51:33         2      39  2018      2   14     4      51
2     2018-02-14 04:52:12         3      40  2018      2   14     4      52
3     2018-02-14 04:52:51         4      40  2018      2   14     4      52
4     2018-02-14 04:53:30         5      40  2018      2   14     4      53
...                   ...       ...     ...   ...    ...  ...   ...     ...
89392 2018-03-21 23:57:24     67740      60  2018      3   21    23      57
89393 2018-03-21 23:58:03     67741      60  2018      3   21    23      58
89394 2018-03-21 23:58:41     67742      60  2018      3   21    23      58
89395 2018-03-21 23:59:20     67743      59  2018      3   21    23      59
89396 2018-03-21 23:59:59     67744      60  2018      3   21    23      59

[89397 rows x 8 columns]


# train test split

In [115]:
# Splitting data into features (X) and target (y)
X = df_no_outliers.drop(["field1", "created_at"], axis=1)  # Features excluding "field1" and "created_at"
y = df_no_outliers["field1"]  # Target variable
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [116]:
# Applying Standard Scaling to the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [117]:
pip install keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [118]:
pip install tensorflow tensorflow-addons kerastuner scikit-learn pandas


  Using cached tensorflow_addons-0.21.0-cp39-cp39-win_amd64.whl (729 kB)



ERROR: Could not find a version that satisfies the requirement kerastuner (from versions: none)
ERROR: No matching distribution found for kerastuner


In [119]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split




In [120]:
# # Define your LSTM model with predefined hyperparameters

# X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
# X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# model = Sequential()
# model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(1, X_train_scaled.shape[1])))
# model.add(LSTM(64, activation='relu'))
# model.add(Dense(1))

# # Compile the model
# optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Adjust the learning rate as needed
# model.compile(optimizer=optimizer, loss='mse')

# # Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model
# history = model.fit(X_train_reshaped, y_train, epochs=50, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# # Evaluate the model on the test set
# y_pred = model.predict(X_test_reshaped)
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error (MSE):", mse)


In [121]:
# # Define your LSTM model with predefined hyperparameters
# model = Sequential()
# model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(1, X_train_scaled.shape[1])))
# model.add(LSTM(64, activation='relu'))
# model.add(Dense(1))

# # Compile the model
# optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Adjust the learning rate as needed
# model.compile(optimizer=optimizer, loss='mse')

# # Define early stopping
# # early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model with more epochs (e.g., 100 epochs)
# history = model.fit(X_train_reshaped, y_train, epochs=100, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# # Evaluate the model on the test set
# y_pred = model.predict(X_test_reshaped)
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error (MSE):", mse)


In [122]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
import kerastuner as kt

# Define a function to build the LSTM model with hyperparameters
def build_model(hp):
    model = keras.Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32),
                   activation=hp.Choice('activation', values=['relu', 'tanh']),
                   return_sequences=True,
                   input_shape=(1, X_train_scaled.shape[1])))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   activation=hp.Choice('activation', values=['relu', 'tanh'])))
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mse',
                  metrics=['mse'])
    return model

# Define the tuner and search space
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of trials to run
    directory='my_tuning_directory',  # Directory to save results
    project_name='my_lstm_tuning'  # Name for this tuning project
)

# Perform hyperparameter tuning
tuner.search(X_train_reshaped, y_train, epochs=50, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
best_model.fit(X_train_reshaped, y_train, epochs=100, validation_data=(X_test_reshaped, y_test), callbacks=[early_stopping])

# Evaluate the best model
y_pred = best_model.predict(X_test_reshaped)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE) for Best Model:", mse)


INFO:tensorflow:Reloading Tuner from my_tuning_directory\my_lstm_tuning\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
2235/2235 [==============================] - 30s 12ms/step - loss: 364.1902 - mse: 364.1902 - val_loss: 247.6577 - val_mse: 247.6577
Epoch 2/100
2235/2235 [==============================] - 25s 11ms/step - loss: 236.6598 - mse: 236.6598 - val_loss: 191.4661 - val_mse: 191.4661
Epoch 3/100
2235/2235 [==============================] - 28s 13ms/step - loss: 182.3184 - mse: 182.3184 - val_loss: 180.8147 - val_mse: 180.8147
Epoch 4/100
2235/2235 [==============================] - 30s 14ms/step - loss: 157.4214 - mse: 157.4214 - val_loss: 141.6537 - val_mse: 141.6537
Epoch 5/100
2235/2235 [==============================] - 28s 13ms/step - loss: 124.0970 - mse: 124.0970 - val_loss: 115.8176 - val_mse: 115.8176
Epoch 6/100
2235/2235 [==============================] - 28s 12ms/step - loss: 101.7576 - mse: 101.7576 - val_loss: 95.8240 - val_mse: 95.8240
Epoch 7/10

In [123]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r_squared = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mse = mean_squared_error(y_test, y_pred)

print("TANK1:")
print("R-squared:", r_squared)
print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("Mean Squared Error:", mse)

TANK1:
R-squared: 0.9963211225424352
Mean Absolute Error: 0.5228424342153323
Root Mean Squared Error: 1.1524682070956076
Mean Squared Error: 1.3281829683661643


In [125]:
# Save the trained model with the desired name
model.save("Final_Tank1_lstmModel.h5")
model.save("Final_Tank1_lstmModel.keras")
# Load the saved model
from tensorflow import keras

loaded_model = keras.models.load_model("Tank1_lstmModel.h5")

# You can then use loaded_model for predictions or further training


C:\Users\MG\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# GRU Model